## Save data on Cassandra
This notebook will save all files from a driving directory into cassandra database

Install driver
```bash
pip install cassandra-driver
```

References:
* https://www.youtube.com/watch?v=SE3A9tUdE8s
* http://datastax.github.io/python-driver/getting_started.html
* http://datastax.github.io/python-driver/object_mapper.html
* http://stupidpythonideas.blogspot.co.uk/2014/07/three-ways-to-read-files.html
* https://www.youtube.com/watch?v=A-j0IShmG1U

In [1]:
from cassandra.cluster import Cluster
import cassandra.util
import uuid
import numpy as np
folder_dataset = "/home/leoara01/work/Hackathon/Data/Wheel_Cams/Track8_Wheel_Cam/"

insert_string = """
    INSERT INTO tb_drive (id, wheel_angle, acc, image)
    VALUES (%s, %s, %s, %s)
    """

In [2]:
# Considering that the cluster is on localhost
cluster = Cluster()
# Other option if you know the IPs
# cluster = Cluster(['192.168.0.1', '192.168.0.2'])
# Get a session to the database
session = cluster.connect('mydb')

### Doing a query

In [3]:
rows = session.execute('SELECT * FROM tb_drive limit 5')
print('Collumns:',rows.column_names)
for row in rows:
    print(row.id, row.acc, row.wheel_angle)

Collumns: ['id', 'acc', 'image', 'wheel_angle']
69689abe-574f-11e7-9708-989096d72294 0.4868069887161255 -0.1188569962978363
cf4fe0ca-574b-11e7-9708-989096d72294 0.5739529728889465 -0.08012589812278748
c1991ef7-574b-11e7-9708-989096d72294 0.0 -0.2931489944458008
7c2bccf6-574e-11e7-9708-989096d72294 1.0 0.0
8c8cfc98-574f-11e7-9708-989096d72294 0.3028320074081421 0.0


In [4]:
result = session.execute('SELECT count(*) as total FROM tb_drive')
print(result[0].total)

5500


### Read Data

In [ ]:
# Lists to hold image filename and label
img_paths = []
steering_list = []
acc_list = []

with open(folder_dataset + "data.txt") as f:
    for line in f:
        # Image path
        img_paths.append(folder_dataset + line.split()[0])                
        
        # Steering wheel label
        steering_list.append(np.float32(line.split()[1]))
        # Acceleration label
        acc_list.append(np.float32(line.split()[2]))

num_images = len(img_paths)
print("Number of images: %d" % num_images)

# Use zip to create a list with images/labels
dataset = list(zip(img_paths, steering_list, acc_list))

Number of images: 880


### Insert Data into Cassandra database

In [ ]:
for data in dataset:
    # Split from dataset the image path, steering angle, and acceleration
    img_path, steering_angle, acc = data 
        
    # Load image (png compressed)
    with open(img_path, 'rb') as f:
        content_file = f.read()

    # Insert into database    
    session.execute(insert_string,(uuid.uuid1(), steering_angle, acc, content_file))    